In [3]:
import cv2
import numpy as np 
import face_recognition
from datetime import datetime
import os
path = 'C:/Users/Nikhi/OneDrive/Pictures/project/New folder'
images = []
className = []
myList = os.listdir(path)
print(myList)

['billgates.jpg', 'elon.jpg', 'elontest.jpg', 'jackma.jpg']


In [34]:
for cl in myList:
    currentImg = cv2.imread(f'{path}/{cl}')
    images.append(currentImg)
    className.append(os.path.splitext(cl)[0])
print(className)   

['billgates', 'elon', 'elontest']


In [35]:
def findencoding(images):
    encodeList=[]
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList    

In [44]:
def mark_Attendence(name):
    with open('attendence.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            date_str = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{date_str}')    

In [37]:
encodeListKnown=findencoding(images)
print('Encoding Completed')

Encoding Completed


In [ ]:
cap = cv2.VideoCapture(0)
while True:
    success, img = cap.read()
    imgS = cv2.resize(img,(0,0),None,0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
     
    facesCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)
    
    for encode_face,face_loc in zip(encodeCurFrame,facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown,encode_face)
        facedist = face_recognition.face_distance(encodeListKnown,encode_face)
        print(facedist)
        matchIndex = np.argmin(facedist)

        if matches[matchIndex]:
            name = className[matchIndex].upper()
            print(name)
            y1,x2,y2,x1=face_loc
            y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            mark_Attendence(name)
        

In [ ]:
cv2.imshow('webcam',img)
cv2.waitKey(1)